Option chains
=======

In [ ]:
from ib_insync import *
from ibapi.common import *
# from ibapi.common import MarketDataTypeEnum
from pandas import *



In [ ]:
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=21)

Suppose we want to find the options on the SPX, with the following conditions:

* Use the next three monthly expiries;
* Use strike prices within +- 20 dollar of the current SPX value;
* Use strike prices that are a multitude of 5 dollar.

To get the current market value, first create a contract for the underlyer (the S&P 500 index):

In [ ]:
# spx = Index('TREX', 'SMART',"USD")
stock = Stock('META', 'SMART', 'USD')
ib.qualifyContracts(stock)

To avoid issues with market data permissions, we'll use delayed data:

In [ ]:
ib.reqMarketDataType(MarketDataTypeEnum.DELAYED)
# ib.reqMarketDataType(4)

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [ ]:
ib.qualifyContracts(stock)
ticker = ib.reqTickers(stock)
ticker = []

print(ticker)

print(type(ticker))
print (len(ticker))
#print( f"Lat Price => {ticker[-1]}")




Take the current market value of the ticker:

In [ ]:
spxValue = ticker.marketPrice()

spxValue

The following request fetches a list of option chains:

In [ ]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

These are four option chains that differ in ``exchange`` and ``tradingClass``. The latter is 'SPX' for the monthly and  'SPXW' for the weekly options. Note that the weekly expiries are disjoint from the monthly ones, so when interested in the weekly options the monthly options can be added as well.

In this case we're only interested in the monthly options trading on SMART:

In [ ]:
chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain

What we have here is the full matrix of expirations x strikes. From this we can build all the option contracts that meet our conditions:

In [ ]:
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P', 'C']

contracts = [Option(stock, expiration, strike, right, 'SMART', tradingClass='SPX')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*contracts)
len(contracts)

In [ ]:
contracts[0]

Now to get the market data for all options in one go:

In [ ]:
#ask
#bid
tickers = ib.reqTickers(*contracts)



tickers[0]

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [ ]:
ib.disconnect()